Kaggle challenge IVADO: https://www.kaggle.com/c/semaine-ia-reddit-challenge
I used TF-IDF representation for words and cleaned the dataset using scipy (tokenization, lematization, lower case and ponctuation). Got about 64% accuracy on the test set provided for the challenge. I tried to oversample my dataset with EDA method https://github.com/jasonwei20/eda_nlp but although it improved the f1_score on my CV (almost 90%), it does not generalize better with unknown data.

Use GPU for faster training

In [135]:
import tensorflow as tf
tf.test.gpu_device_name()

''

Load the driver helper and mount

In [136]:
# Load the driver helper and mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [137]:
!ls '/content/drive/My Drive/ML/Kaggle_IVADO/'

eda_train_sampling2.csv  preds2.csv  preds6.csv      train.csv
eda_train_sampling3.csv  preds3.csv  preds.csv
eda_train_sampling.csv	 preds4.csv  preds_last.csv
Kaggle_IVADO_2020.ipynb  preds5.csv  test.csv


LOAD DATASET

In [157]:
import numpy as np
import pandas as pd
from collections import Counter

def loadDataset():
  #Load Packages
  df_submission = pd.read_csv("/content/drive/My Drive/ML/Kaggle_IVADO/test.csv")
  df = pd.read_csv("/content/drive/My Drive/ML/Kaggle_IVADO/train.csv")
  #df = pd.read_csv("/content/drive/My Drive/ML/Kaggle_IVADO/eda_train_sampling3.csv")
  return df_submission, df

In [158]:
df_submission, df = loadDataset()
#print(df)
print(df_submission)

        id                                            comment
0    14053  that s an interesting idea  but i think i d ha...
1    14872  off to debtor s prison with you  meet your cel...
2    26742  now all i am thinking is a normal sized facele...
3    26927  if they do reproduce  or if more dragon eggs a...
4    15099  gt  debunked this assertion with data showing ...
..     ...                                                ...
938  10489  forever grateful i graduated with minimal stud...
939  27765  i wish i could tell you that thomas fought the...
940  19911  if he is that good with mario  just imagine wh...
941  10459  gt  castle made shoddy cheeses for almost 30 y...
942  20096  you can emulate this incredibly easily if you ...

[943 rows x 2 columns]


In [159]:
from sklearn import preprocessing
def convert(data):
  number = preprocessing.LabelEncoder()
  data['comment'] = number.fit_transform(data['comment'])
  data=data.fillna(-999) # fill holes with default value
  return data


In [160]:
# Load dataset
dataset = np.array(df)
dataset2 = np.array(df_submission)
print(dataset.shape)
X = dataset[:,0]
Y = dataset[:,1]

X_ = dataset2[:,1]
#from sklearn.preprocessing import LabelEncoder
#label_encoder = LabelEncoder()
#Y = label_encoder.fit_transform(Y)
print(X[0])
print(X_[0])
print(Y.shape)
#print(sorted(Counter(Y).items()))

(3770, 2)
would have been a million times more exciting if the living characters were nt shown  and jon opened his eyes at the end of the teaser 
that s an interesting idea  but i think i d have a really hard time taking it seriously and pretending to be nice to a virtual child when i knew that it could nt really understand me  especially in future sessions when i knew that the switch was going to happen 
(3770,)


TRAINING USEFUL METHODS

In [161]:
!pip3 install pyriemann
from pyriemann.utils.viz import plot_confusion_matrix
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,precision_recall_fscore_support
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

In [162]:
#General confusion matrix that will be used for KNN and SVM
def PlotPrectionConfLvl(preds,X,Y,model):
  # make predictions on the testing data
  print("[INFO] predicting confusion level ...")
  acc         = np.mean(preds == Y)
  print("Classification accuracy: %f " % (acc))

  # plot the confusion matrices for both classifiers
  names        = ["Conf0", "Conf1", "Conf2", "Conf3"]
  #names        = possible_labels
  plt.figure(0)
  plot_confusion_matrix( Y, preds, names, title = str(type(model))+'\nAccuracy:{0:.3f}'.format(acc))
  plt.show()

In [163]:
#Confusion matrix that will be used for LSTM
def PlotPrectionConfLvlLSTM(preds,X,Y,model):
  # make predictions on the testing data
  print("[INFO] predicting confusion level ...")
  preds_       = preds.argmax(axis = -1)
  acc         = np.mean(preds_ == Y.argmax(axis=-1))
  print("Classification accuracy: %f " % (acc))

  # plot the confusion matrices for both classifiers
  names        = ["Conf0", "Conf1", "Conf2", "Conf3"]
  #names        = possible_labels
  plt.figure(0)
  plot_confusion_matrix(Y.argmax(axis = -1), preds_, names, title = str(type(model))+'\nAccuracy:{0:.3f}'.format(acc))
  plt.show()

In [164]:
def Reshape2D(X):
  print(np.array(list(X)).dtype)
  print(X.shape)
  X4 = X.reshape(X.shape[0],X.shape[1]*X.shape[2])
  print(X4.shape)
  #printList(X3)
  return X4

def Reshape3D(X):
  print(np.array(list(X)).dtype)
  print(X.shape)
  X4 = X.reshape(X.shape[0],1,X.shape[1])
  print(X4.shape)
  #printList(X3)
  return X4

In [165]:
def PlotLearningCurveLSTM(lstm):
  #print(lstm.history.keys())
  # summarize history for accuracy
  plt.plot(lstm.history['accuracy'])
  plt.plot(lstm.history['val_accuracy'])
  plt.plot(lstm.history['loss'])
  plt.plot(lstm.history['val_loss'])
  plt.title('model accuracy')
  #plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train accuracy', 'test accuracy','loss','validation loss'], loc='upper left')
  plt.show()

USE SPICY TO PRE-PROCESS THE DATA

In [166]:
!pip install spacy
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


Tokenizing the Text: Separate each word from sentences, Cleaning Text Data: Removing Stopwords, Lexicon Normalization: a way of processing words that reduces them to their roots

In [167]:
import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

Remove spaces and converts text into lowercase

In [168]:
from sklearn.base import TransformerMixin
# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

In [169]:
my_doc = X

clean_txt = []
for sentence in my_doc:
  clean_txt.append(clean_text(sentence))
print(clean_txt[0],clean_txt[1])

would have been a million times more exciting if the living characters were nt shown  and jon opened his eyes at the end of the teaser question for those in the know  i ve bought fallout 4 but have nt gotten to it yet i ve in fact made a point to finish fallout 3  and just beat the main quest last night  now i m onto the side missions and dlc  based on fallout 4  do people think this is going to be worth it  i ve seen reviews of the base game  and they seem mostly positive  but still note some issues with the game


In [170]:
# Create list of word tokens
token_list = []
for sentence in clean_txt:
  token_list.append(spacy_tokenizer(sentence))
print(token_list[0],token_list[1])

['million', 'times', 'exciting', 'living', 'characters', 'nt', 'shown', 'jon', 'opened', 'eyes', 'end', 'teaser'] ['question', 'know', 've', 'bought', 'fallout', '4', 'nt', 'gotten', 've', 'fact', 'point', 'finish', 'fallout', '3', 'beat', 'main', 'quest', 'night', 'm', 'missions', 'dlc', 'based', 'fallout', '4', 'people', 'think', 'going', 'worth', 've', 'seen', 'reviews', 'base', 'game', 'positive', 'note', 'issues', 'game']


Vectorization Feature Engineering (TF-IDF)

In [171]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

X2 = np.array([', '.join(x) for x in token_list])

encoder = preprocessing.LabelEncoder()
Y2 = encoder.fit_transform(df['subreddit'].values)
X_train, X_test, Y_train, Y_test = train_test_split(X2,Y2, test_size = 0.2, random_state = 42)

print(X_train.shape)


from sklearn.pipeline import make_pipeline
pipe = make_pipeline(
    CountVectorizer(),
    TfidfTransformer(),
)

X_train = pipe.fit_transform(X_train.ravel()) 
X_test = pipe.transform(X_test)
X_test2 = pipe.transform(X_)

(3016,)


TRAINING

In [172]:
from sklearn.metrics import classification_report, f1_score, make_scorer

def classification_report_with_f1_score(y_true, y_pred):
    print (f1_score(y_true, y_pred, average='macro'))
    print (classification_report(y_true, y_pred)) # print classification report
    return f1_score(y_true, y_pred, average='macro') # return f1 score

KNN

In [173]:
#import the KNeighborsClassifier from sklearn
from sklearn.neighbors import KNeighborsClassifier

#import metrics model to check the accuracy
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV, TimeSeriesSplit
from imblearn.pipeline import Pipeline, make_pipeline

def KNNCV(X_trainn,Y_trainn,X_test,Y_test,k_range):
  #Try running from k=1 through 25 and record testing accuracy
  score = {}
  scores_list = []
  
  #HYPERPARAMETER TUNING WITH CV
  for k in k_range:


   # kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state = 42)
    
    knn = KNeighborsClassifier(n_neighbors=k)

    #OVERSAMPLING
    #imba_pipeline = make_pipeline(SMOTE(random_state=42), knn)
        
    print("Score for k=",k)

   # nested_score = cross_validate(knn, X_trainn, Y_trainn, cv=kfold, \
    #           scoring=make_scorer(classification_report_with_f1_score),return_train_score=True)

    knn.fit(X_train, Y_train)
    y_pred=knn.predict(X_test)
    score = f1_score(Y_test,y_pred, average='macro')

    scores_list.append(score)
    print("y_pred: ",y_pred)

    print("done for k=",k)

    print(score)
    print(classification_report(Y_test,y_pred))
    
    #print("RESULTS: ")
    #print("full res: ",nested_score)
    #mean = nested_score["test_score"].mean()
    #print("mean CV for this k", mean)
    #scores_list.append(mean)
    #print("mean CV for all k", scores_list)
    #print("STD for this k", nested_score["test_score"].std()*2)


  return scores_list,knn,y_pred

In [174]:
k_range = [23]
scores_list,knn,y_pred = KNNCV(X_train,Y_train,X_test,Y_test,k_range)

Score for k= 23
y_pred:  [0 1 5 2 2 4 4 2 1 2 0 0 5 2 0 3 3 2 4 4 1 1 3 3 0 2 2 0 2 4 1 4 0 2 1 0 0
 1 1 5 3 0 2 4 2 4 0 2 3 2 0 0 3 3 4 4 2 4 3 1 4 0 1 0 1 4 2 2 2 2 4 4 0 1
 2 3 0 3 2 4 4 3 1 4 5 0 4 4 3 0 4 1 4 2 4 0 3 0 2 4 0 0 4 0 4 4 1 3 1 0 4
 2 4 4 0 4 0 4 3 2 3 4 0 0 2 0 2 2 0 4 0 4 4 4 4 3 2 4 3 0 2 2 3 4 4 3 3 0
 4 3 1 3 2 2 2 1 2 3 3 0 4 0 3 2 4 2 4 4 1 4 2 2 2 3 2 2 4 3 5 0 2 4 0 0 2
 5 4 4 4 3 0 2 4 4 1 2 4 2 2 2 3 4 3 2 4 1 2 2 1 1 4 4 3 5 2 4 2 2 0 2 4 2
 2 2 0 3 5 0 3 2 2 0 2 4 0 2 2 1 4 0 5 2 1 3 3 3 4 1 2 2 2 2 4 2 4 2 4 1 5
 5 3 4 0 4 0 4 5 4 4 2 4 4 2 4 1 4 2 3 2 4 3 2 4 2 4 2 4 0 3 4 4 4 1 4 4 4
 3 2 4 4 5 2 3 1 2 5 0 3 5 4 0 4 2 3 0 2 1 1 1 3 2 5 0 1 4 4 4 0 2 2 2 4 2
 4 0 0 2 2 4 2 4 3 2 1 3 0 4 4 3 2 0 3 1 2 0 2 2 4 3 3 2 4 0 0 2 3 1 4 0 3
 2 0 5 1 2 4 2 4 1 4 4 2 4 2 2 4 2 0 4 2 1 0 4 0 2 1 2 0 2 4 0 1 1 0 3 2 0
 0 1 4 4 2 2 2 4 2 5 4 3 1 5 2 5 0 0 4 5 4 2 4 1 4 3 4 4 4 2 1 0 1 2 2 2 4
 0 2 2 0 0 0 4 4 3 5 4 3 1 2 2 1 0 2 4 0 1 4 4 1 2 2 0 2 2 4 2 2 1 0 2 4 4


In [ ]:
print(y_pred.shape)
preds = encoder.inverse_transform(y_pred)
print(preds)
np.savetxt("/content/drive/My Drive/ML/Kaggle_IVADO/preds.csv", preds, delimiter=",",fmt='%s')

(7540,)
['news' 'gameofthrones' 'gaming' ... 'gameofthrones' 'science' 'news']


In [ ]:

preds = knn.predict(X_test2)
preds = encoder.inverse_transform(preds)
print(preds)
np.savetxt("/content/drive/My Drive/ML/Kaggle_IVADO/preds.csv", preds, delimiter=",",fmt='%s')

LSTM

In [175]:
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [176]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [177]:
import re
from keras.preprocessing.sequence import pad_sequences

# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 50
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(token_list)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X2 = tokenizer.texts_to_sequences(token_list)
X2 = pad_sequences(X2, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X2.shape)

X2_ = tokenizer.texts_to_sequences(df_submission['comment'].values)
X2_ = pad_sequences(X2_, maxlen=MAX_SEQUENCE_LENGTH)

encoder = LabelEncoder()

Y2 = encoder.fit_transform(df['subreddit'].values)

print(X2.shape)
print(X2_.shape)
print(Y2.shape)

#gameofthrones = 1
#gaming = 2
# funny = 0
# news = 3
# politics = 4
# science = 5

Found 13827 unique tokens.
Shape of data tensor: (3770, 50)
(3770, 50)
(943, 50)
(3770,)


In [178]:
X_train, X_test, Y_train, Y_test = train_test_split(X2,Y2, test_size = 0.3, random_state = 42)
#X_train, Y_train = SmoteSampling(X_train, Y_train)
#X_train,Y_train = NaiveSampling(X_train,Y_train)

print(Y_train)
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

[1 2 0 ... 0 0 0]
(2639, 50)
(2639, 6)
(1131, 50)
(1131, 6)


In [179]:
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, SpatialDropout1D, LSTM, Embedding, TimeDistributed, Dropout, GaussianNoise, Activation
from keras.callbacks import ModelCheckpoint

def LSTMModel(X_train, X_valid, Y_train, Y_valid):
  nb_classes = Y_train.shape[1]
  epochs = 15
  batch_size = 64
  hidden_size = 200

  model = Sequential()
  model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_train.shape[1]))
  model.add(SpatialDropout1D(0.2))
  model.add(LSTM(hidden_size, dropout=0.2, recurrent_dropout=0.2))
  model.add(Dense(nb_classes, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc',f1_m,precision_m, recall_m])

  history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_valid, Y_valid), verbose=1, callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])
  
  model2 = model
  y_pred = model2.predict(X_valid, batch_size=batch_size, verbose=1)
  y_pred_=np.argmax(y_pred, axis=1)
  Y_valid_ = np.argmax(Y_valid, axis=1)
  print(classification_report(Y_valid_,y_pred_))

  return model,X_train, X_valid, Y_train, Y_valid,y_pred,history

In [ ]:
lstm,Xtrain,Xvalid,Ytrain,Yvalid,ypred,history = LSTMModel(X_train, X_test, Y_train, Y_test)

Epoch 1/15
42/42 [==============================] - 17s 405ms/step - loss: 1.7399 - acc: 0.2330 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 1.7055 - val_acc: 0.2520 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 2/15
42/42 [==============================] - 17s 402ms/step - loss: 1.5703 - acc: 0.3717 - f1_m: 0.0341 - precision_m: 0.2704 - recall_m: 0.0205 - val_loss: 1.5280 - val_acc: 0.3837 - val_f1_m: 0.0512 - val_precision_m: 0.6463 - val_recall_m: 0.0269
Epoch 3/15
42/42 [==============================] - 17s 400ms/step - loss: 1.1072 - acc: 0.5911 - f1_m: 0.3371 - precision_m: 0.8060 - recall_m: 0.2284 - val_loss: 1.2307 - val_acc: 0.5270 - val_f1_m: 0.4226 - val_precision_m: 0.7621 - val_recall_m: 0.2942
Epoch 4/15
42/42 [==============================] - 17s 397ms/step - loss: 0.6205 - acc: 0.7870 - f1_m: 0.7428 - precision_m: 0.8778 - recall_m: 0.6469 - val_loss: 1.1684 - val_acc: 0.5747 - val_f1_m: 0.54

In [ ]:
preds = lstm.predict(X2_)
preds = np.argmax(preds, axis=1)
#gameofthrones = 1
#gaming = 2
# funny = 0
# news = 3
# politics = 4
# science = 5
predictions = []
for pred in preds:
  if pred == 0:
    predictions = np.append(predictions,'funny')
  elif pred == 1:
    predictions = np.append(predictions,'gameofthrones')
  elif pred == 2:
    predictions = np.append(predictions,'gaming')
  elif pred == 3:
    predictions = np.append(predictions,'news')
  elif pred == 4:
    predictions = np.append(predictions,'politics')
  elif pred == 5:
    predictions = np.append(predictions,'science')
  else:
    print("error")
print(predictions)
np.savetxt("/content/drive/My Drive/ML/Kaggle_IVADO/preds2.csv", predictions, delimiter=",",fmt='%s')

SVM MODEL

Vectorization TF-IDF

In [182]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

X2 = np.array([', '.join(x) for x in token_list])

encoder = preprocessing.LabelEncoder()
Y2 = encoder.fit_transform(df['subreddit'].values)
X_train, X_test, Y_train, Y_test = train_test_split(X2,Y2, test_size = 0.2, random_state = 42)

print(X_train.shape)


from sklearn.pipeline import make_pipeline
pipe = make_pipeline(
    CountVectorizer(),
    TfidfTransformer(),
)

X_train = pipe.fit_transform(X_train.ravel()) 
X_test = pipe.transform(X_test)
X_test2 = pipe.transform(X_)

(3016,)


In [183]:
from sklearn.svm import SVC
def SVM(X_train, X_test, Y_train, Y_test):
  classifier = SVC(C=2.01, kernel='linear', degree=3, gamma='auto')
  classifier.fit(X_train, Y_train)
  y_pred = classifier.predict(X_test)
  cr = classification_report(Y_test,y_pred)
  score = f1_score(Y_test,y_pred, average='macro')
  print(score)
  print(cr)
  return classifier,y_pred

In [184]:
svm, ypred = SVM(X_train, X_test, Y_train, Y_test)

0.669200997287071
              precision    recall  f1-score   support

           0       0.54      0.58      0.56       127
           1       0.83      0.71      0.76        92
           2       0.64      0.74      0.69       166
           3       0.55      0.65      0.60       124
           4       0.79      0.68      0.73       176
           5       0.78      0.58      0.67        69

    accuracy                           0.67       754
   macro avg       0.69      0.66      0.67       754
weighted avg       0.68      0.67      0.67       754



In [ ]:
preds = svm.predict(X_test2)
preds = encoder.inverse_transform(preds)
#print(preds)
np.savetxt("/content/drive/My Drive/ML/Kaggle_IVADO/preds_last.csv", preds, delimiter=",",fmt='%s')

In [185]:
from sklearn.linear_model import LogisticRegression
def LR(X_train, X_test, Y_train, Y_test):
  classifier = LogisticRegression()
  classifier.fit(X_train, Y_train)
  y_pred = classifier.predict(X_test)
  cr = classification_report(Y_test,y_pred)
  score = f1_score(Y_test,y_pred, average='macro')
  print(score)
  print(cr)
  return classifier,y_pred

In [186]:
logreg, ypred = LR(X_train, X_test, Y_train, Y_test)

0.6559052815752969
              precision    recall  f1-score   support

           0       0.62      0.48      0.54       127
           1       0.91      0.64      0.75        92
           2       0.66      0.81      0.73       166
           3       0.51      0.73      0.60       124
           4       0.75      0.76      0.75       176
           5       0.93      0.41      0.57        69

    accuracy                           0.67       754
   macro avg       0.73      0.64      0.66       754
weighted avg       0.70      0.67      0.67       754



In [187]:
preds = logreg.predict(X_test2)
preds = encoder.inverse_transform(preds)
#print(preds)
np.savetxt("/content/drive/My Drive/ML/Kaggle_IVADO/preds.csv", preds, delimiter=",",fmt='%s')

In [188]:
from sklearn.ensemble import RandomForestClassifier
def RF(X_train, X_test, Y_train, Y_test):
  classifier = RandomForestClassifier(n_estimators=1000,max_depth=100, random_state=0)
  classifier.fit(X_train, Y_train)
  y_pred = classifier.predict(X_test)
  cr = classification_report(Y_test,y_pred)
  score = f1_score(Y_test,y_pred, average='macro')
  print(score)
  print(cr)
  return classifier,y_pred

In [189]:
rf, ypred = RF(X_train, X_test, Y_train, Y_test)

0.634535029475987
              precision    recall  f1-score   support

           0       0.41      0.68      0.51       127
           1       0.90      0.66      0.76        92
           2       0.70      0.74      0.72       166
           3       0.52      0.60      0.56       124
           4       0.87      0.64      0.73       176
           5       0.84      0.38      0.52        69

    accuracy                           0.64       754
   macro avg       0.71      0.62      0.63       754
weighted avg       0.70      0.64      0.65       754

